# Twitter Dogs Data Wrangling

In this project we will retrieve data from twitter using an archive file that contains tweets ID's.

Then, we will clean at least eight data quality issues and two tidiness issues in our dataset.



In [42]:
#Import modules, packages

import requests
import json
import pandas as pd
import numpy as np
import tweepy
import time

## Gathering Data

We will collect data from three sources:
1. Archived tweets - A file we downloaded and is in our computer files
2. Image predictions - A file that is on a server and we need to get it through a GET request and save it to a local file
3. API tweets - Data obtained from the Twitter API using tweet ID's from the Archived tweets files. One all requests are made, we store the data in a file

We will gather data in the order above .

### Archived Tweets

Since we have the file in our local directory, we only need to load it to a pandas dataframe.

In [110]:
# load the archive file
df_arch = pd.read_csv('twitter-archive-enhanced.csv')
df_arch.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [127]:
# Check size of the archive file
print('Rows: {}, Columns: {}'.format(df_arch.shape[0],df_arch.shape[1]))

Rows: 2356, Columns: 17


### Image Predictions

In order to obtain the image predictions file, we will need to put a GET request to a server and parse the HTTP response to extract the file content. Then, we can store it to a file and then read it into a pandas dataframe.

In [12]:
# retrieve image prediction file from a server using requests
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)

In [13]:
# write the http response content-text into a file
with open('image-predictions.tsv','w') as file:
    file.write(r.text)

In [128]:
# read image predictions file into a dataframe
df_pred = pd.read_csv('image-predictions.tsv', sep='\t')
df_pred.head(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### API Tweets

Finally, we want to gather data from the Twitter API. In order to do this, we will need to have a twitter account and create a developer account. Once, the developer account has been created one can create an app and this will give a set of codes to interact with Twitter API. The codes are consumer_key, consumer_secret, access_token and access_token_secret. Such codes, allow us to programatically ask request to the twitter API. Then, we will need to use the tweepy library to create an object that interacts with the twitter API and grants us access using the codes mentioned above.
Once we have granted access to put request to the Twitter API, we can use the archived tweets ID's to ask for information regarding such tweets. 

>**Note:** Be aware that the Twitter API has rate limits, which means after a certain amount fo request it will force a delay wait time before a new request is replied. You can automate the program to wait setting to True the parameters `wait_on_rate_limit` and `wait_on_rate_limit_notify` when creating an instance of the tweepy.API.

**Important:** Never share in Github or anywhere public your `consumer_key`, `consumer_secret`, `access_token` and `access_token_secret` !!!

In [21]:
# Obtaining the authorization keys from a file to avoid sharing them publicly
with open('twitterAPI-keys.txt','r') as json_file:
    twitter_keys = json.load(json_file)

consumer_key = twitter_keys['consumer']['key']
consumer_secret = twitter_keys['consumer']['secret']

access_token = twitter_keys['access']['token']
access_token_secret = twitter_keys['access']['token_secret']

In [41]:
# obtaining access to the Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Configuring the twitter API to use our keys and to automatically wait when the rate limit is reached
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [81]:
# example on how to retrieve a tweet using an ID
tweet_id = 666020888022790149
tweet = api.get_status(tweet_id, tweet_mode='extended')
print(tweet)

Status(geo=None, in_reply_to_screen_name=None, in_reply_to_user_id_str=None, user=User(contributors_enabled=False, id=4196983835, profile_banner_url='https://pbs.twimg.com/profile_banners/4196983835/1525830435', id_str='4196983835', is_translator=False, _json={'profile_image_url_https': 'https://pbs.twimg.com/profile_images/948761950363664385/Fpr2Oz35_normal.jpg', 'following': False, 'default_profile_image': False, 'created_at': 'Sun Nov 15 21:41:29 +0000 2015', 'profile_sidebar_fill_color': '000000', 'contributors_enabled': False, 'statuses_count': 8536, 'id': 4196983835, 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'is_translator': False, 'notifications': False, 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/4196983835/1525830435', 'id_str': '4196983835', 'protected': False, 'profile_link_color': 'F5ABB5', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'verified': True, 'profile_use_backg

In [84]:
# Accessing the tweet status JSON object gotten from Twitter API response
tweet._json

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Nov 15 22:32:08 +0000 2015',
 'display_text_range': [0, 131],
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/BLDqew2Ijj',
    'expanded_url': 'https://twitter.com/dog_rates/status/666020888022790149/photo/1',
    'id': 666020881337073664,
    'id_str': '666020881337073664',
    'indices': [108, 131],
    'media_url': 'http://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg',
    'sizes': {'large': {'h': 720, 'resize': 'fit', 'w': 960},
     'medium': {'h': 720, 'resize': 'fit', 'w': 960},
     'small': {'h': 510, 'resize': 'fit', 'w': 680},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'type': 'photo',
    'url': 'https://t.co/BLDqew2Ijj'}],
  'symbols': [],
  'urls': [],
  'user_mentions': []},
 'extended_entities': {'media': [{'display_url': 'pic.twitter.com/BLDqew2Ijj',
    'expanded_url': 'https://twitter.com/dog_

In [93]:
# Example dictionary of how to extract the desired fields from the tweet status JSON object
t = { 'id' : tweet._json['id_str'],
        'favorite_count' : tweet._json['favorite_count'],
        'retweeted': tweet._json['retweeted'],
        'retweet_count': tweet._json['retweet_count'],
        'full_text': tweet._json['full_text'],
        'media_url': tweet._json['entities']['media'][0]['media_url'],
        'source': tweet._json['source'] }
t

{'favorite_count': 2560,
 'full_text': 'Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj',
 'id': '666020888022790149',
 'media_url': 'http://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg',
 'retweet_count': 513,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'}

In [105]:
# Functions to retrieve a tweet and store the desired data from it into a dictionary

def getTweetfromID(tweet_id):
    """
    Function to retrieve a tweet from Twitter API using the tweet ID
    
    parameters:
        @tweet_id: tweet ID to use to retrieve a tweet
        
    return:
        @tweet: the tweet retrieved, and None if there was no tweet retrieved
    """
    tweet = None
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
    except tweepy.TweepError as e:
        # Example format [{"message":"Sorry, that page does not exist","code":34}]
        #print(e.__dict__) # full http response
        print('Exception for tweet ID {}: {} '.format(tweet_id, e))
    return tweet


def storeTweetdata(tweet, tweet_dict):
    """
    Function to store the desired tweet data from a tweet status JSON object
    
    parameters:
        @tweet: tweet status JSON object obtained from twitter API
        @tweet_dict: dictionary in which the data is getting stored
        
    return:
        @tweet_dict: returns the same dictionary in which the data is getting stored
    """
    tweet_json = tweet._json
    
    tweet_dict['tweet_id'].append(tweet_json['id_str'])
    tweet_dict['favorite_count'].append(tweet_json['favorite_count'])
    tweet_dict['retweeted'].append(tweet_json['retweeted'])
    tweet_dict['retweet_count'].append(tweet_json['retweet_count'])
    tweet_dict['full_text'].append(tweet_json['full_text'])
    
    if 'media' in tweet_json['entities']:
        tweet_dict['media_url'].append(tweet_json['entities']['media'][0]['media_url'])
    else:
        tweet_dict['media_url'].append(np.nan)
        
    tweet_dict['source'].append(tweet_json['source'])
    
    return tweet_dict


In [106]:
# retrieving and storing all tweets from Twitter API using archived files tweet_id

batch = 1
BATCH_SIZE = 180
initial_id = 1
last_id = 0


tweet_dict = { 'tweet_id' : [],
                'favorite_count' : [],
                'retweeted': [],
                'retweet_count': [],
                'full_text': [],
                'media_url': [],
                'source': [] 
                }

start = time.process_time()
for count_id, tweet_id in enumerate(df_arch['tweet_id'],1):
    
    last_id = count_id
    
    # get tweet
    tweet = getTweetfromID(tweet_id)
    
    # store tweet
    if tweet is not None:
        tweet_dict = storeTweetdata(tweet, tweet_dict)
        
    # display statistics
    if count_id%BATCH_SIZE == 0:
        
        end = time.process_time()
        ellapsed_time = end - start
        print('\n--------- Batch {} IDs: {}-{}, Ellapsed time: {} ---------\n'.\
              format(batch, initial_id, last_id, ellapsed_time))
        batch +=1
        intial_id = last_id
        start = time.process_time()

end = time.process_time()
ellapsed_time = end - start
print('\n--------- Batch {} IDs: {}-{}, ellapsed time: {} ---------\n'.\
      format(batch, initial_id, last_id, ellapsed_time))


Exception for tweet ID 888202515573088257: [{'message': 'No status found with that ID.', 'code': 144}] 
Exception for tweet ID 873697596434513921: [{'message': 'No status found with that ID.', 'code': 144}] 
Exception for tweet ID 869988702071779329: [{'message': 'No status found with that ID.', 'code': 144}] 
Exception for tweet ID 866816280283807744: [{'message': 'No status found with that ID.', 'code': 144}] 
Exception for tweet ID 861769973181624320: [{'message': 'No status found with that ID.', 'code': 144}] 

--------- Batch 1 IDs: 1-180, Ellapsed time: 4.3216719999999995 ---------

Exception for tweet ID 845459076796616705: [{'message': 'No status found with that ID.', 'code': 144}] 
Exception for tweet ID 842892208864923648: [{'message': 'No status found with that ID.', 'code': 144}] 
Exception for tweet ID 837012587749474308: [{'message': 'No status found with that ID.', 'code': 144}] 

--------- Batch 2 IDs: 1-360, Ellapsed time: 4.325434000000001 ---------

Exception for twe

Rate limit reached. Sleeping for: 652


Exception for tweet ID 754011816964026368: [{'message': 'No status found with that ID.', 'code': 144}] 

--------- Batch 6 IDs: 1-1080, Ellapsed time: 4.394260999999993 ---------


--------- Batch 7 IDs: 1-1260, Ellapsed time: 4.348782000000007 ---------


--------- Batch 8 IDs: 1-1440, Ellapsed time: 4.342611000000005 ---------


--------- Batch 9 IDs: 1-1620, Ellapsed time: 4.388689999999997 ---------



Rate limit reached. Sleeping for: 686



--------- Batch 10 IDs: 1-1800, Ellapsed time: 4.370533999999999 ---------


--------- Batch 11 IDs: 1-1980, Ellapsed time: 4.436374999999998 ---------


--------- Batch 12 IDs: 1-2160, Ellapsed time: 4.388532999999995 ---------


--------- Batch 13 IDs: 1-2340, Ellapsed time: 4.381236999999999 ---------


--------- Batch 14 IDs: 1-2356, ellapsed time: 0.38497099999999307 ---------



In [107]:
# read tweets data obtained from the API into dataframe
df_api = pd.DataFrame(tweet_dict)
df_api.head(5)

,favorite_count,full_text,media_url,retweet_count,retweeted,source,tweet_id
0,38520,This is Phineas. He's a mystical boy. Only eve...,http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,8499,False,"<a href=""http://twitter.com/download/iphone"" r...",892420643555336193
1,33027,This is Tilly. She's just checking pup on you....,http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,6247,False,"<a href=""http://twitter.com/download/iphone"" r...",892177421306343426
2,24861,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,4139,False,"<a href=""http://twitter.com/download/iphone"" r...",891815181378084864
3,41920,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,8615,False,"<a href=""http://twitter.com/download/iphone"" r...",891689557279858688
4,40068,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,9354,False,"<a href=""http://twitter.com/download/iphone"" r...",891327558926688256


In [108]:
# Store data into csv file
df_api.to_csv('tweet_json.txt', index=False)

### Data Gathering Checkpoint

The data has been stored in their corresponding files, so that one can read the data from the files.

In [146]:
# read tweets API file into dataframe
df_api = pd.read_csv('tweet_json.txt')
df_api.head(5)

,favorite_count,full_text,media_url,retweet_count,retweeted,source,tweet_id
0,38520,This is Phineas. He's a mystical boy. Only eve...,http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,8499,False,"<a href=""http://twitter.com/download/iphone"" r...",892420643555336193
1,33027,This is Tilly. She's just checking pup on you....,http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,6247,False,"<a href=""http://twitter.com/download/iphone"" r...",892177421306343426
2,24861,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,4139,False,"<a href=""http://twitter.com/download/iphone"" r...",891815181378084864
3,41920,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,8615,False,"<a href=""http://twitter.com/download/iphone"" r...",891689557279858688
4,40068,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,9354,False,"<a href=""http://twitter.com/download/iphone"" r...",891327558926688256


## Assessing Data

Once the data was gathered, we can start assessing the data and checking what is dirty and what is messy.

### Visual Assessment

We start using pandas, but if not all data is shown we will need to use a spreadsheet software such as Google sheets.

#### Archived Tweets

In [114]:
df_arch

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


#### Image Predictions Dataset

In [129]:
df_pred

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


Checking for a `tweet_id` declared as desktop in the prediction, but actually being a dog looking at a PC screen.

In [153]:
df_arch[df_arch['tweet_id'] == 666268910803644416]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2337,666268910803644416,NaN,NaN,2015-11-16 14:57:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Very concerned about fellow dog trapped in com...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666268910...,10,10,None,None,None,None,None


Checking for a tweet_id declared as guinea pig in the prediction and actually being a guinea pig.

In [154]:
df_arch[df_arch['tweet_id'] == 666362758909284353]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2330,666362758909284353,NaN,NaN,2015-11-16 21:10:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Unique dog here. Very small. Lives in containe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666362758...,6,10,None,None,None,None,None


In [155]:
df_arch[df_arch['tweet_id'] == 666362758909284353]['text']

2330    Unique dog here. Very small. Lives in containe...
Name: text, dtype: object

Apparently someone wanted to cheat **WeRateDogs** and it might be good to see if this id exists in the tweeter data gotten from API, otherwise this tweet was deleted.

In [156]:
df_api[df_api['tweet_id'] == 666362758909284353]

,favorite_count,full_text,media_url,retweet_count,retweeted,source,tweet_id
2317,778,Unique dog here. Very small. Lives in containe...,http://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg,568,False,"<a href=""http://twitter.com/download/iphone"" r...",666362758909284353


In [166]:
df_api[df_api['tweet_id'] == 666362758909284353]['full_text']

2317    Unique dog here. Very small. Lives in containe...
Name: full_text, dtype: object

The tweet was not deleted despite it was not a dog, maybe we should check another tweet of a non-dog picture

In [157]:
df_api[df_api['tweet_id'] == 666411507551481857]

,favorite_count,full_text,media_url,retweet_count,retweeted,source,tweet_id
2313,445,This is quite the dog. Gets really excited whe...,http://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg,325,False,"<a href=""http://twitter.com/download/iphone"" r...",666411507551481857


Ok another tweet of a non-dog picture exists in the tweets retrived from the API, which means it was not deleted. This makes more of a mystery why we have missing records in our dataset retrieved from the API. However, this also will need to be listed as quality issue since we only want dogs data and not other animals or things.

#### Tweets from API

In [151]:
df_api

,favorite_count,full_text,media_url,retweet_count,retweeted,source,tweet_id
0,38520,This is Phineas. He's a mystical boy. Only eve...,http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,8499,False,"<a href=""http://twitter.com/download/iphone"" r...",892420643555336193
1,33027,This is Tilly. She's just checking pup on you....,http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,6247,False,"<a href=""http://twitter.com/download/iphone"" r...",892177421306343426
2,24861,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,4139,False,"<a href=""http://twitter.com/download/iphone"" r...",891815181378084864
3,41920,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,8615,False,"<a href=""http://twitter.com/download/iphone"" r...",891689557279858688
4,40068,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,9354,False,"<a href=""http://twitter.com/download/iphone"" r...",891327558926688256
5,20089,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,3105,False,"<a href=""http://twitter.com/download/iphone"" r...",891087950875897856
6,11765,Meet Jax. He enjoys ice cream so much he gets ...,http://pbs.twimg.com/media/DF1eOmZXUAALUcq.jpg,2061,False,"<a href=""http://twitter.com/download/iphone"" r...",890971913173991426
7,65083,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,18831,False,"<a href=""http://twitter.com/download/iphone"" r...",890729181411237888
8,27624,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,4255,False,"<a href=""http://twitter.com/download/iphone"" r...",890609185150312448
9,31720,This is Cassie. She is a college pup. Studying...,http://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,7379,False,"<a href=""http://twitter.com/download/iphone"" r...",890240255349198849


### Programatic Assessment

In [111]:
df_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [130]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [152]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2343 entries, 0 to 2342
Data columns (total 7 columns):
favorite_count    2343 non-null int64
full_text         2343 non-null object
media_url         2068 non-null object
retweet_count     2343 non-null int64
retweeted         2343 non-null bool
source            2343 non-null object
tweet_id          2343 non-null int64
dtypes: bool(1), int64(3), object(3)
memory usage: 112.2+ KB


In [160]:
all_columns = pd.Series(list(df_arch) + list(df_pred) + list(df_api))
all_columns[all_columns.duplicated()]

17    tweet_id
34      source
35    tweet_id
dtype: object

In [159]:
df_arch.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [161]:
df_api.isnull().sum()

favorite_count      0
full_text           0
media_url         275
retweet_count       0
retweeted           0
source              0
tweet_id            0
dtype: int64

In [136]:
df_arch.name.value_counts()

None         745
a             55
Charlie       12
Oliver        11
Cooper        11
Lucy          11
Tucker        10
Penny         10
Lola          10
Winston        9
Bo             9
Sadie          8
the            8
Toby           7
an             7
Daisy          7
Buddy          7
Bailey         7
Bella          6
Scout          6
Rusty          6
Milo           6
Leo            6
Koda           6
Dave           6
Jack           6
Stanley        6
Jax            6
Oscar          6
Bentley        5
            ... 
Kayla          1
Lance          1
Mya            1
Jennifur       1
Bert           1
Halo           1
Sojourner      1
Stormy         1
Thor           1
Luther         1
Harnold        1
Tom            1
Dudley         1
Geoff          1
Bronte         1
Tug            1
Jiminus        1
Staniel        1
Flurpson       1
Devón          1
Link           1
Zooey          1
Edd            1
Beemo          1
Amélie         1
Bruno          1
Dook           1
Skittle       

In [137]:
df_arch.tweet_id.duplicated().sum()

0

In [140]:
df_arch.in_reply_to_status_id.value_counts()

6.671522e+17    2
8.562860e+17    1
8.131273e+17    1
6.754971e+17    1
6.827884e+17    1
8.265984e+17    1
6.780211e+17    1
6.689207e+17    1
6.658147e+17    1
6.737159e+17    1
7.590995e+17    1
8.862664e+17    1
7.384119e+17    1
7.727430e+17    1
7.468859e+17    1
8.634256e+17    1
6.693544e+17    1
6.914169e+17    1
6.920419e+17    1
6.753494e+17    1
7.291135e+17    1
8.406983e+17    1
6.747400e+17    1
7.501805e+17    1
6.744689e+17    1
7.638652e+17    1
6.747934e+17    1
8.503288e+17    1
6.747522e+17    1
8.816070e+17    1
               ..
8.380855e+17    1
8.211526e+17    1
8.558616e+17    1
8.558585e+17    1
7.032559e+17    1
6.678065e+17    1
8.018543e+17    1
7.667118e+17    1
6.855479e+17    1
6.717299e+17    1
6.715610e+17    1
6.758457e+17    1
6.924173e+17    1
7.476487e+17    1
8.381455e+17    1
6.903413e+17    1
8.476062e+17    1
8.352460e+17    1
6.813394e+17    1
8.795538e+17    1
6.860340e+17    1
8.571567e+17    1
6.765883e+17    1
7.044857e+17    1
8.707262e+

### Quality


**`twitter-archive-enhanced`** table
* Several recordsets that have no value in the name column as expressed as _None_
* Several recordsets have in the name column values that are not a proper name such as _a_,_an_,_my_ or _the_ 
    * Tweet_id 887517139158093824 has name column value _such_
    * Tweet_id 666411507551481857 has name column value _quite_
    * Tweet_id 666407126856765440 has name column value _a_


* Contains tweets that are not dogs (`image-predictions` table can help find the non-dogs tweets)
* Source column has a full html text instead of a single word describing the source
* Column in_reply_to_status_id has 2278 missing values
* Column in_reply_to_user_id has 2278 missing values
* Column retweeted_status_id has 2175 missing values
* Column retweeted_status_user_id has 2175 missing values
* Column retweeted_status_timestamp has 2175 missing values


**`image-predictions`** table
* p1,p2,p3,p4 have breed names in different capitalization formats ([PROPER CAPITALIZATION](https://www.quickanddirtytips.com/education/grammar/capitalizing-dog-breeds))
* Missing records (2075 instead of 2546) - not sure if we can fix or need to drop records
* Some records are not dogs

**`tweet_json`** table
* Missing records (2343 instead of 2546)
* Erroneous datatype for tweet_id


### Tidiness

**`twitter-archive-enhanced`** table
* Columns doggo, floofer, pupper, and puppo can be combined into a single type column


**`image-predictions`** table
* p1,p2,p3,and p4 can be make a column prediction and their values and breed prediction can be three new columns probability, breed_predicted and if dog or not
* Image column doesn't need to be part of this table, keeping tweet_id is sufficient


**`tweet_json`** table
* This table should be part of the `twitter-archive-enhanced` table. 
* Three columns from `tweet_json` are repeated in `twitter-archive-enhanced`: **media_url**, **source** and **full_text**. __[Better to keep the archive data since `tweet_json` does not have the complete information of these columns]__



## Cleaning Data

Once we have assessed the data, we have a better idea of what needs to be cleaned. 

We have a lot of missing data, but we do not have additional data sources, so it will be a good idea to start from fixing tidiness issues as tidy datasets tend to be easier to clean. 

First, we need to copy our datasets into a clean version of each, this to be able to look at how they originally are at any point in time. This can help us compare the cleaning process to the original dirty and messy data.

In [368]:
arch_clean = df_arch.copy()

In [356]:
pred_clean = df_pred.copy()

In [357]:
api_clean = df_api.copy()

### Tidiness

##### `twitter-archive-enhanced`: Columns doggo, floofer, pupper, and puppo can be combined into a single type column

**Define**

Doggo, floofer, pupper and puppo are all a **stage**, so we can create a single column using pandas melt method and keeping only the non-duplicated values. In order to distinguish the None values for each class of stage, we can make doggo contain Unknown instead of None. This can be done using map(). Then, we will need to query for columns different than None and finally drop duplicated tweet_id records with stage None and a different valid stage.

We will use stage as the column name and the value on each column doggo, floofer, pupper and puppo that corresponds to the tweet. Otherwise, we will fill it with Unknown.

**Code**

In [369]:
# columns of arch_clean
arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [370]:
# change the None fields in doggo for Unknown, 
# this will help identify the fields needed to keep for the stage after melt 
arch_clean['doggo'] = arch_clean['doggo'].map({'None':'Unknown', 'doggo':'doggo'})

In [371]:
# Creating the new column and its values
arch_clean = pd.melt(arch_clean, id_vars=['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name'],
                    value_vars=['doggo', 'floofer', 'pupper', 'puppo'],
                    var_name='stages',
                    value_name='stage')

In [372]:
# selecting the Unknown to leave the repeated None values behind
arch_clean = arch_clean.query('stage != "None"')

In [373]:
# checking for duplicate tweet_id to remove
arch_clean.tweet_id.duplicated().sum()

297

In [374]:
# drop duplicated tweet_id's 
arch_clean = arch_clean.drop_duplicates(subset='tweet_id', keep='last', inplace=False)

# drop the stages column that contains the column name 
arch_clean.drop('stages', axis=1, inplace=True)

**Test**

In [375]:
# check the columns after fixing the tidiness
arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'stage'],
      dtype='object')

In [376]:
# check the size is same as original dataset
print('Dataset after tidiness fix: {}'.format(arch_clean.shape))
print('Dataset before tidiness: {}'.format(df_arch.shape))

Dataset after tidiness fix: (2356, 14)
Dataset before tidiness: (2356, 17)


In [377]:
# check the unique values
arch_clean.stage.unique()

array(['Unknown', 'doggo', 'floofer', 'pupper', 'puppo'], dtype=object)

In [366]:
# check the count values for stage
arch_clean.stage.value_counts()

Unknown    1976
pupper      257
doggo        83
puppo        30
floofer      10
Name: stage, dtype: int64

In [272]:
# verify there are no null values
arch_clean.stage.isnull().sum()

0

##### `image-predictions`: p1, p2, p3, and p4 can be a column prediction and their values and breed prediction can be three new columns probability, breed_predicted and if dog or not

**Define**

use column level

**Code**

In [379]:
# check columns
pred_clean.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

In [381]:
pred_clean = pd.melt(pred_clean, id_vars=['tweet_id', 'jpg_url', 'img_num'],
                        value_vars=['p1_conf', 'p1_dog', 'p2_conf', 'p2_dog', 'p3_conf', 'p3_dog' ],
                        var_name='prediction')
                        #value_name=['prob', 'isDog'])

In [382]:
pred_clean

,tweet_id,jpg_url,img_num,prediction,value
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,p1_conf,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,p1_conf,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,p1_conf,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,p1_conf,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,p1_conf,0.560311
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,p1_conf,0.651137
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,p1_conf,0.933012
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,p1_conf,0.692517
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,p1_conf,0.962465
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,p1_conf,0.201493


In [ ]:
pd.melt(arch_clean, id_vars=['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name'],
                    value_vars=['doggo', 'floofer', 'pupper', 'puppo'],
                    var_name='stages',
                    value_name='stage')

**Test**

### Missing data

### Quality